# Wave Glider Level 3b processor
## smooth, subsample, save subsets; primarily met and vel


## About Level 3a and Level 3b data

Level 3a data: (L3a_all)
- The only difference from Level 2 is that NaNs are removed by interpolation
- This is a "wave resolving" data set with modest QC (removal/interpolation of gaps<1 sec)

Level 3a subsets: (just interpolate gaps<1 sec and extract individual data sets)
- Make a velocity subset (L3a_vel)
- make a wave subset (L3a_wave)
- make a met subset (L3a_met)

Level 3b met data:
- Same as Level 3a but variables are averaged to 1 min
- 20 Hz variables are subsampled to 1 Hz time base
- Maybe in the future, will want to subsample all variables to a 1 min time base
- May not produce for all variables (eg, vechiles with met packages off)

Level 3b velocity data:
- Velocity data and averaged to 5 minutes

## This notebook is for level 3b data
1. Load L3a data files
1. Do L3b processing (smooth, subsample, save subsets; primarily met and vel)



In [1]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import matplotlib
import datetime as dt
################
# This allows us to import Tom_tools
import sys
sys.path.append('../../Tom_tools/') # you may need to adjust this path
# sys.path.append('../SWOT_IW_SSH/jtf/Tom_tools/') # you may need to adjust this path
import Tom_tools_v1 as tt
################

In [2]:
# %matplotlib inline
%matplotlib qt 
savefig = False # set to true to save plots as file
plt.rcParams['figure.figsize'] = (5,4)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 400
plt.close('all')
plt.rcParams['axes.xmargin'] = 0
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

__figdir__ = '../plots/WG_timeseries/'
savefig_args = {'bbox_inches':'tight', 'pad_inches':0.2}
plotfiletype='png'

In [9]:
# Names of L3a dataS
'''
# Same as L2, but with small gaps filled
new_file = path_out+WG+'_L3a.nc'
# Write 1 Hz data to netcdf file
new_file = path_out+WG+'_L3a_1Hz_met.nc'
# Write 20 Hz data to netcdf file
new_file = path_out+WG+'_L3a_20Hz_met.nc'
# Write Workhorse ADCP data to netcdf file
new_file = path_out+WG+'_L3a_ADCP.nc'
# Write wave data to netcdf file
new_file = path_out+WG+'_L3a_waves.nc'
'''


"\n# Same as L2, but with small gaps filled\nnew_file = path_out+WG+'_L3a.nc'\n# Write 1 Hz data to netcdf file\nnew_file = path_out+WG+'_L3a_1Hz_met.nc'\n# Write 20 Hz data to netcdf file\nnew_file = path_out+WG+'_L3a_20Hz_met.nc'\n# Write Workhorse ADCP data to netcdf file\nnew_file = path_out+WG+'_L3a_ADCP.nc'\n# Write wave data to netcdf file\nnew_file = path_out+WG+'_L3a_waves.nc'\n"

In [4]:
# path = '/mnt/e/SMODE_data/pilot/WG/L2/'
path = '/mnt/d/SMODE_data/pilot/WG/L3a/'
path_out = '/mnt/d/SMODE_data/pilot/WG/L3b/'

WG = 'WHOI43'#'Kelvin'#'Stokes'#

file =  WG+'_L3a.nc'
# file_20Hz = 'SMODE_PFC_Wavegliders_'+WG+'_L3a_20Hz_met.nc'
# file_1Hz = 'SMODE_PFC_Wavegliders_'+WG+'_L3a_1Hz_met.nc'
# file_adcp = 'SMODE_PFC_Wavegliders_'+WG+'_L3a_ADCP.nc'

In [5]:
%%time
ds = xr.open_dataset(path+file, engine = 'netcdf4', decode_times = True) #decode_times = False, 

CPU times: user 371 ms, sys: 194 ms, total: 565 ms
Wall time: 1.13 s


https://xarray-test.readthedocs.io/en/latest/indexing.html#dataset-indexing

```
ds = arr.to_dataset(name='foo')
ds.isel(space=[0], time=[0])
```

In [67]:
ds.isel('time_1Hz'=10)

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (2155847494.py, line 1)

In [66]:
ds['time_1Hz']=ds.time_1Hz[0:-10:]

ValueError: conflicting sizes for dimension 'time_1Hz': length 771961 on 'latitude_1Hz' and length 771951 on {'time_1Hz': 'time_1Hz'}

In [7]:
def make_var_list(ds_in,time_coord):
    """
    Find all the variables with a given time coord
 
    Parameters
    ----------
    ds_in : xarray.dataset
    time_coord : str

    Returns
    -------
    result : list of str
        list of vars meeting criterion
    """
    
    var_list = []  
    not_used = []  
    for var in ds_in.data_vars:
        try:
            if ds_in.data_vars.get(var).dims[0]==time_coord:
                var_list.append(var)
                print(var)
        except:
            not_used.append(var)

    return var_list

In [ ]:
def remove_nans(var_list, ds_in):
    """
    remove all nans in variables in an input xarray.dataset
 
    Parameters
    ----------
    ds_in : xarray.dataset
    var_list : list of strings

    Returns
    -------
    result : revised xarray.dataset
    """
    #t = ds_in[ds_in.data_vars.get(var).coords.dims[0]]
    #t_noninterp = t[ff]

    for var in var_list:
        var_raw = ds_in.data_vars.get(var).copy()
        ff = np.flatnonzero(np.isnan(var_raw)==0)
        var_nanfree = var_raw[ff]

        numnans = np.size(np.flatnonzero(np.isnan(var_raw)))
        locals()[var] = var_raw.rename(var) #locals()['string'] makes a variable with the name string
        print(var+' created, '+str(np.round(100*(1-numnans)/np.size(t),3))+'% of values are NaN'+', number of nans=' + str(numnans))
        locals()[var].values = var_nanfree
        ds_in[var] = locals()[var]
    return ds_new

In [8]:
def subset(var_list, ds_in):

    ds_new = ds_in[var_list]
    return ds_new

In [9]:
def add_vars(var_list, ds_in, ds_out):
    '''
    Copies variables in var_list from ds_in to ds_out.  
    '''
    var_existing = []
    for var in ds_in.data_vars:
        var_existing.append(var)

    #ds_out = ds_out[var_existing]
    ds_out[var_list] = ds_in[var_list].copy()
    return ds_out

In [10]:
'''
ds_new = xr.Dataset().assign_attrs(ds.attrs)  # make empty xr.Dataset but copy attributes from original file
ds_WH = xr.Dataset().assign_attrs(ds.attrs)  # make empty xr.Dataset but copy attributes from original file
'''

'\nds_new = xr.Dataset().assign_attrs(ds.attrs)  # make empty xr.Dataset but copy attributes from original file\nds_WH = xr.Dataset().assign_attrs(ds.attrs)  # make empty xr.Dataset but copy attributes from original file\n'

In [11]:
# ds_new = add_vars(var_list, ds, ds_new)

In [12]:
# Write data to netcdf file
'''
new_file = path_out+WG+'_L3a.nc'
print ('saving to ', new_file)
ds_new.to_netcdf(path=new_file)
ds_new.close()
print ('finished saving')

# Write Workhorse ADCP data to netcdf file
new_file = path_out+WG+'_L3a_ADCP.nc'
print ('saving to ', new_file)
ds_WH.to_netcdf(path=new_file)
ds_WH.close()
print ('finished saving')
'''


"\nnew_file = path_out+WG+'_L3a.nc'\nprint ('saving to ', new_file)\nds_new.to_netcdf(path=new_file)\nds_new.close()\nprint ('finished saving')\n\n# Write Workhorse ADCP data to netcdf file\nnew_file = path_out+WG+'_L3a_ADCP.nc'\nprint ('saving to ', new_file)\nds_WH.to_netcdf(path=new_file)\nds_WH.close()\nprint ('finished saving')\n"

In [13]:
# compare different time vectors to decide how to line up variables from different time bases
# ds.time_20Hz
# ds.time_1Hz
# ds.Workhorse_time

# Try the simple thing
# xy, x_ind, y_ind = np.intersect1d(ds.time_1Hz, ds.time_20Hz, assume_unique=True, return_indices=True)
# That doesn't work (no surprise) because the times don't line up exactly (possibly numerical precision issue)

# I think they are already nearly lined up
ds.time_1Hz[0:10:1]-ds.time_20Hz[0:200:20]
# so, try 0,20,40... in 20 Hz array

tfoo = ds.time_20Hz[0:-1:20]
#tfoo and time_1Hz are very close, within 10^-4 sec



In [14]:
nsec=60

In [52]:
# ds.time_20Hz variables
# wdir is positive clockwise from North
# need to make U, V
wind_speed_low = tt.run_avg1d(ds.wind_speed, nsec*20)
wind_east_low = tt.run_avg1d(ds.wind_east,nsec*20)
wind_north_low = tt.run_avg1d(ds.wind_north,nsec*20)
pitch_low = tt.run_avg1d(ds.pitch,nsec*20)
roll_low = tt.run_avg1d(ds['roll'],nsec*20)

In [53]:
# ds.time_1Hz variables
WXT_U=ds.WXT_wind_speed*np.cos(ds.WXT_wind_direction/np.pi/2)
WXT_V=ds.WXT_wind_speed*np.sin(ds.WXT_wind_direction/np.pi/2)
WXT_U_low = tt.run_avg1d(WXT_U,nsec)
WXT_V_low = tt.run_avg1d(WXT_V,nsec)
WXT_wspd_low = tt.run_avg1d(ds.WXT_wind_speed,nsec)
WXT_atmp_low = tt.run_avg1d(ds.WXT_air_temperature,nsec)
WXT_rh_low = tt.run_avg1d(ds.WXT_relative_humidity,nsec)
swr_low = tt.run_avg1d(ds.SMP21_shortwave_flux,nsec)
lwr_low = tt.run_avg1d(ds.SGR4_longwave_flux,nsec)
lat_low = tt.run_avg1d(ds.latitude_1Hz,nsec)
lon_low = tt.run_avg1d(ds.longitude_1Hz,nsec)

In [54]:
# Workhorse_time variables
U_low=tt.run_avg2d(ds.Workhorse_vel_east,nsec,1)
V_low=tt.run_avg2d(ds.Workhorse_vel_north,nsec,1)
W_low=tt.run_avg2d(ds.Workhorse_vel_up,nsec,1)

In [55]:
# Subsample smoothed 20 Hz vars to 1 Hz
wind_speed_low = wind_speed_low[0::20]
wind_east_low = wind_east_low[0::20]
wind_north_low = wind_north_low[0::20]
pitch_low = pitch_low[0::20]
roll_low = roll_low[0::20]


In [56]:
np.shape(wind_speed_low)
np.shape(wind_east_low)


(771961,)

(771961,)

In [57]:
fig, axs = plt.subplots(1, 1)
fig.autofmt_xdate()

plt.plot(ds.time_1Hz,pitch_low)
plt.plot(ds.time_1Hz,roll_low)
plt.title(str(nsec/60)+'-min average pitch and roll')
plt.ylabel('Degrees')
plt.ylim(-2,2)

if savefig:
    plt.savefig(__figdir__+WG+'_mean_pitch_roll' + '.' +plotfiletype,**savefig_args)


Text(0.5, 1.0, '1-hr average pitch and roll')

Text(0, 0.5, 'Degrees')

(-2.0, 2.0)

In [59]:
fig, axs = plt.subplots(1, 1)
fig.autofmt_xdate()

plt.plot(ds.time_1Hz,wind_speed_low)
plt.plot(ds.time_1Hz,WXT_wspd_low)
plt.title(str(nsec/60)+'-min average wind speed')
plt.ylabel('Degrees')
#plt.ylim(-2,2)

if savefig:
    plt.savefig(__figdir__+WG+'_mean_wspd' + '.' +plotfiletype,**savefig_args)


Text(0.5, 1.0, '1.0-min average wind speed')

Text(0, 0.5, 'Degrees')

In [40]:
'''
new_file = path_out+WG+'_L3b_met.nc'
print ('saving to ', new_file)
ds_new.to_netcdf(path=new_file)
ds_new.close()
print ('finished saving')
'''

"\nnew_file = path_out+WG+'_L3b_met.nc'\nprint ('saving to ', new_file)\nds_new.to_netcdf(path=new_file)\nds_new.close()\nprint ('finished saving')\n"

In [22]:
WG

'WHOI43'

In [62]:
# Plot spectra of 'raw' L3a wind speed
fig, axs = plt.subplots(1, 1)
M=151
wspd=ds.wind_speed.values
wspd_WXT=ds.WXT_wind_speed.values
tt.spectrum_band_avg(wspd,1/20,M,winstr=None,plotflag=True,ebarflag=None)
tt.spectrum_band_avg(wspd_WXT,1,M,winstr=None,plotflag=True,ebarflag=False)
tt.spectrum_band_avg(wind_speed_low,1,M,winstr=None,plotflag=True,ebarflag=False)
plt.title('Gill, WXT wind speed')
plt.xlabel('Hz')
plt.ylabel('Spectral density [m$^2$/s$^2$/Hz]')
plt.legend(['Gill Wind speed','WXT wind speed'],loc='upper right')
if savefig:
    plt.savefig(__figdir__+WG+'_wsp_spectra' + '.' +plotfiletype,**savefig_args)


(array([1.86137239e+04, 1.33838061e+02, 7.82490941e+01, ...,
        3.15836523e-03, 4.45316618e-03, 8.30519391e-03]),
 array([9.84506906e-05, 2.94056668e-04, 4.89662645e-04, ...,
        9.99947601e+00, 9.99967162e+00, 9.99986722e+00]),
 302.0)

(array([2.13317756e+04, 1.30148896e+02, 6.84812920e+01, ...,
        1.21065851e-01, 1.16002843e-01, 1.35281962e-01]),
 array([9.84505694e-05, 2.94056306e-04, 4.89662043e-04, ...,
        4.99479896e-01, 4.99675502e-01, 4.99871107e-01]),
 302.0)

(array([nan, nan, nan, ..., nan, nan, nan]),
 array([9.84505694e-05, 2.94056306e-04, 4.89662043e-04, ...,
        4.99479896e-01, 4.99675502e-01, 4.99871107e-01]),
 302.0)

Text(0.5, 1.0, 'Gill, WXT wind speed')

Text(0.5, 0, 'Hz')

Text(0, 0.5, 'Spectral density [m$^2$/s$^2$/Hz]')

## Preparing for flux computation

Inputs for COARE 3.5:  
    u = ocean surface relative wind speed (m/s) at height zu(m)  
    t = bulk air temperature (degC) at height zt(m)  
    rh = relative humidity (%) at height zq(m)  
    ts = sea water temperature (degC) - see jcool below  
    P = surface air pressure (mb) (default = 1015)  
    Rs = downward shortwave radiation (W/m^2) (default = 150)  
    Rl = downward longwave radiation (W/m^2) (default = 370)  
    zu = wind sensor height (m) (default = 18m)  
    zt = bulk temperature sensor height (m) (default = 18m)  
    zq = RH sensor height (m) (default = 18m)  
    lat = latitude (default = 45 N)  
    zi = PBL height (m) (default = 600m)  
    rain = rain rate (mm/hr)  
    cp = phase speed of dominant waves (m/s)  
    sigH =  significant wave height (m)  
    jcool = cool skin option (default = 1 for bulk SST)  
    
Note: I don't see an input for SST depth--> That's because there is no Warm Layer correction in this version of the code.  

Here is a link to an older matlab version that does have it:  
https://github.com/carsonwitte/Falkor-DWL-Code  

It wouldn't be that hard to add the WL correction-- basically, estimate fluxes, estimate WL correction, and re-estimate fluxes.  


  
  
Required inputs:  
u	zu	t	zt	rh	zq	P	ts	Rs	Rl	lat	zi	rain	cp	sigH


This could be a good way to save the output, but maybe not:  

```
A = coare35vn(u, ta, rh, ts, P=Pa, Rs=rs, Rl=rl, zu=16, zt=16, zq=16,
                lat=Lat, zi=ZI, rain=Rain, jcool=1)
fnameA = os.path.join(path,'test_35_output_py_04022022.txt')
A_hdr = 'usr\ttau\thsb\thlb\thlwebb\ttsr\tqsr\tzot\tzoq\tCd\t'
A_hdr += 'Ch\tCe\tL\tzet\tdter\tdqer\ttkt\tRF\tCdn_10\tChn_10\tCen_10'
np.savetxt(fnameA,A,fmt='%.18e',delimiter='\t',header=A_hdr)
```
